### Problem definition
### Put itens in a knapsack without exceed your capacity and obtaining maximum profit

In [12]:
# Imports
from gurobipy import *
import numpy as np

In [13]:
# Read Data
def read_instance(path):
    with open(path) as r:
        num_itens = int(r.readline()) 
        r.readline() # empty line between data
        profits = [int(x) for x in r.readline().split()]
        r.readline()
        capacity = int(r.readline())
        r.readline()
        weights = [int(x) for x in r.readline().split()]

    return num_itens, profits, capacity, weights

### In this code we implement two algorithms
### Relaxed Dantzing, to obtain an upper bound, and
### Viable Dantzing, to obtain a lower bound.
### And a MIP model with gurobi api

In [22]:
def relaxed_dantzig(num_itens, profits, capacity, weights):
    '''
        Ordene em ordem não-crescente os itens de acordo com a relação pi/wi
        Insira itens na mochila tal forma que não exceda a capacidade.
        
        a solução relaxada não é inteira, é apenas [0, 1] logo
        x = [0, 0.4, 0, 1], por exemplo

        K é o conjunto de itens na mochila tal que
        o somatório para todo i pertencente a K, wi <= c
        
        definimos xh como item de parada, tal que
        (o somatório para todo i pertencente a K, wi) + wk > c 
        
        xk pertence a (0, 1)
    '''
    # values of itens is relation pi/wi
    values = [profits[i] / weights[i] for i in range(num_itens)]
    # Sort index in reversed order
    arg_sort_reversed = np.argsort(np.array(values))[::-1]

    # initialization of variables
    K = np.zeros(num_itens)
    x = np.zeros(num_itens)
    curr_capacity = 0

    # Fill Knapsack
    for i in arg_sort_reversed:
        # If knapsack has capacity to iten, put on
        if weights[i] + curr_capacity <= capacity:
            K[i] = 1
            x[i] = 1
            curr_capacity += weights[i]
        # Else, put on maximum possible 
        else:
            x[i] = (capacity - curr_capacity) / weights[i]
    # Calculate the cost of solution
    obj_value = x @ np.array(profits)
    return x, obj_value

In [33]:
def viable_dantzig(num_itens, profits, capacity, weights):
    '''
        Similar to relaxed dantzing, but the solution is binary,
        for garantee the viability.
    '''
    # Relaxed dantzig solution
    x, _ = relaxed_dantzig(num_itens, profits, capacity, weights) 
    
    # Transform all index in integer
    x = np.floor(x)
    # Calculate the cost of solution
    obj_value = x @ np.array(profits)
    return x, obj_value

In [40]:
def gurobi_knapsack(num_itens, profits, capacity, weights):
    # Create a model
    knap = Model()
    # Add variables to model, relaxed [0, 1]
    x = knap.addVars(num_itens, lb=0, ub=1, name='x')
    # Add capacity constrain to model
    knap.addConstr((x.prod(weights)) <= capacity, name='knapsack')
    # Set a objective function to model, as Maximize the profit
    knap.setObjective(x.prod(profits), GRB.MAXIMIZE)
    # Solve the model
    knap.optimize()

### Go to tests!

In [37]:
num_itens, profits, capacity, weights = read_instance('data/instances_knapsack/10/10_100_1.txt')

In [38]:
x, obj_value = relaxed_dantzig(num_itens, profits, capacity, weights)
print('x :', list(x))
print('Objetive Function Value :', obj_value)

x : [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.16666666666666666, 1.0, 1.0]
Objetive Function Value : 308.33333333333337


In [39]:
x, obj_value = viable_dantzig(num_itens, profits, capacity, weights)
print('x :', list(x))
print('Objetive Function Value :', obj_value)

x : [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0]
Objetive Function Value : 306.0


In [41]:
gurobi_knapsack(num_itens, profits, capacity, weights)

Using license file C:\Users\rodri\gurobi.lic
Academic license - for non-commercial use only - expires 2021-02-01
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1 rows, 10 columns and 10 nonzeros
Model fingerprint: 0xfd168613
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
Presolve time: 0.02s
Presolved: 1 rows, 10 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0900000e+03   6.375000e+00   0.000000e+00      0s
       1    3.0833333e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.04 seconds
Optimal objective  3.083333333e+02
